In [ ]:
from wilcoxon import geo
from wilcoxon import sheets
from wilcoxon import spiderman
from math import sin, cos, atan2, radians, sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot
from abc import ABC, abstractmethod
import pyarrow.parquet as pq
from shapely.ops import transform
from functools import partial
import shapely
import re
import fiona
import json
import requests
import operator
import os
import datetime as dt
import time
import pyproj

class DistanceCalculator:
    
    @staticmethod
    def get_distance_between(point1, point2):
        R = 6373
        lat1 = radians(point1.y)
        long1 = radians(point1.x)
        lat2 = radians(point2.y)
        long2 = radians(point2.x)
        dlat = lat2 - lat1
        dlong = long2 - long1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlong / 2)**2
        return round(R*2*atan2(sqrt(a), sqrt(1-a)), 3)
    
def get_planning():
    PATH_TO_PLANNING_AREAS = "../Geospatial/2022/subzone-census-2010/Subzone_Census2010.kml"
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    planning = gpd.read_file(PATH_TO_PLANNING_AREAS, driver='KML')
    planning["SubzoneCode"] = planning.Description.str.extract("Subzone Code.*?<td>(.*?)</td>")
    planning["Planning"] = planning.Description.str.extract("Planning Area Name.*?<td>(.*?)</td>")
    planning["PlanningCode"] = planning.Description.str.extract("Planning Area Code.*?<td>(.*?)</td>")
    planning["Region"] = planning.Description.str.extract("Region Name.*?<td>(.*?)</td>")
    planning["RegionCode"] = planning.Description.str.extract("Region Code.*?<td>(.*?)</td>")
    planning = planning.rename(columns={"Name":"Subzone"})
    planning = planning.drop("Description",axis=1)
    planning = planning[['Region', 'RegionCode', 'Planning', 'PlanningCode', 'Subzone', 'SubzoneCode', 'geometry']]
    return planning

file_names = os.listdir("../Geospatial/2022/city=Singapore")
grab = pd.concat(
    [pq.read_table("../Geospatial/2022/city=Singapore/"+file_name).to_pandas()
     for file_name in file_names])
grab["rawlng"] = grab.rawlng.round(5)
grab["rawlat"] = grab.rawlat.round(5)
grab["speed"] = grab.speed.round(0).astype(int)
grab["accuracy"] = grab.accuracy.round(0).astype(int)
grab = grab.drop(["bearing", "driving_mode"], axis=1)
grab["osname"] = grab.osname.map({"android": 0, "ios": 1})
grab["trj_id"] = grab.trj_id.astype(int)
grab = grab.sort_values(by=["trj_id","pingtimestamp"])
# grab = grab[(grab.speed >= 0) & (grab.accuracy < 100)]
grab = gpd.GeoDataFrame(grab, geometry=gpd.points_from_xy(grab.rawlng, grab.rawlat))
grab = grab.drop(["accuracy", "rawlat", "rawlng"], axis=1)
grab.columns = ["id", "os", "time", "speed", "geometry"]
grab = grab.set_index("id")

print("ok")
grab_line = grab.groupby("id").agg({
    "os": max,
    "time": (max, min),
    "geometry": (lambda x: shapely.geometry.LineString(x.tolist()), lambda x: x.iloc[0], lambda x: x.iloc[-1])})

grab_line["distance"] = grab_line.geometry["<lambda_0>"].apply(lambda x: x.length*111.33)
grab_line["os"] = grab_line.os["max"]
grab_line["start_time"] = grab_line.time["min"]
grab_line["end_time"] = grab_line.time["max"]
grab_line["geometry1"] = grab_line.geometry["<lambda_0>"]
grab_line["start_point"] = grab_line.geometry["<lambda_1>"]
grab_line["end_point"] = grab_line.geometry["<lambda_2>"]
grab_line = grab_line[["os", "start_time", "start_point", "end_time", "end_point", "geometry1", "distance"]]
grab_line_gdf = gpd.GeoDataFrame(grab_line, geometry="geometry1")
# grab_line_gdf["start_point"] = grab_line_gdf.start_point.apply(lambda x: x.coords[0])
# grab_line_gdf["end_point"] = grab_line_gdf.end_point.apply(lambda x: x.coords[-1])

def find_max_dist(line):
    max_dist = 0
    c_coord = shapely.geometry.Point(line.coords[0])
    for coord in line.coords:
        p_coord = shapely.geometry.Point(coord)
        max_dist = max(max_dist, DistanceCalculator.get_distance_between(p_coord, c_coord))
        c_coord = p_coord
    return max_dist

max_dists = []
for line in tqdm(lines):
    max_dists.append(find_max_dist(line))

grab_line_gdf["max_dist"] = max_dists
grab = grab[grab_line_gdf["max_dist"]<0.1]
grab_line_gdf = grab_line_gdf[grab_line_gdf["max_dist"] < 0.1]
grab_subset = grab.iloc[::10]

planning = get_planning()
planningm = planning[["PlanningCode", "SubzoneCode", "geometry"]].values.tolist()
planning_dict = planning.set_index("SubzoneCode").to_dict("index")
points = grab_subset.geometry.tolist()

points_in_planning = []
for point in tqdm(points):
    not_done = True
    for p_code, s_code, geom in planningm:
        if point.within(geom):
            points_in_planning.append(s_code)
            not_done = False
            break
    if not_done:
        points_in_planning.append(np.nan)
        print("not ok")
grab_subset["plan"] = points_in_planning
grab_subset = grab_subset[~pd.isna(grab_subset.plan)]
grab_subset["time"] = grab_subset.time.apply(lambda x: dt.datetime.fromtimestamp(x))
grab_subset["day"] = grab_subset.time.apply(lambda x: x.day)
grab_subset["w_day"] = grab_subset.time.apply(lambda x: x.weekday())
grab_subset["hour"] = grab_subset.time.apply(lambda x: x.hour)
grab_subset["time"] = grab_subset.time.apply(str)
grab_line_gdf["day"] = grab_line_gdf["start_time"].apply(lambda x: dt.datetime.fromtimestamp(x).day)
grab_line_gdf["hour"] = grab_line_gdf["start_time"].apply(lambda x: dt.datetime.fromtimestamp(x).hour)
grab_line_gdf["weekday"] = grab_line_gdf["start_time"].apply(lambda x: dt.datetime.fromtimestamp(x).weekday())
grab_line_gdf["duration"] = (grab_line_gdf.end_time.apply(dt.datetime.fromtimestamp) - grab_line_gdf.start_time.apply(dt.datetime.fromtimestamp)).apply(lambda x: round(x.seconds/60, 2))
grab_line_gdf["speed"] = grab_line_gdf["distance"] / grab_line_gdf.duration
grab_line_gdf["s_distance"] = grab_line_gdf[["start_point","end_point"]].apply(lambda x: DistanceCalculator.get_distance_between(x.start_point, x.end_point), axis=1)
grab_line_gdf["efficiency"] = grab_line_gdf.s_distance / grab_line_gdf["distance"]
grab_line_gdf["t_efficiency"] = grab_line_gdf.s_distance / grab_line_gdf.duration
grab_line_gdf.drop(["start_point","end_point"], axis=1).to_file("../Geospatial/2022/grab-line-computed.geojson")

def getMRT():
    """
    PURPOSE (NO LONGER WORKS FULLY)

    I made this code to extract all the kinds of data you'd want to know about MRT and LRT stations.
    More specific information will be given in comments below but as of now, here is the link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQIXKejfbHe2FuPKp0qvdS6OfLDceHGO2WvxGgJtyuoD6HyAmd9Qxj8NZsJA4JItvPcqRKyMJUdDVF-/pub?gid=1134364680&single=true&output=pdf
    Do note that this is not the full version of the dataset, as I wanted to only keep the essentials in there.

    Download data for train station exits: TrainStationExit_Jan2020/TrainStationExit06032020.shp
    Download data for train stations: TrainStation_Jan2020/MRTLRTStnPtt.shp
    Download data for OriginDestinationTrain: origin_destination_train_202103.csv
    """
    # path_to_TrainStationExits = ""
    # path_to_TrainStations = ""
    # path_to_OriginDestinationTrain = ""


    print("Extracting Wikipedia links of MRT and LRT stations.")
    """
    PURPOSE

    DOES NOT WORK NOW BECAUSE WIKIPEDIA CHANGED THE FORMAT OF THE PAGE.
    But what this section does is extract the links to all the Wikipedia pages of the respective MRT and LRT stations.
    Changi Airport, Expo, Tanah Merah and Damai LRT were added manually because they were not found anywhere within the landing page.
    """
    stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations")
    stations.getTables()
    stationLinksM = stations.tables[1]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
    stationLinksM = [x for x in stationLinksM if ("a"+x)[-1] == "n"]
    stationLinksM.extend(["https://en.wikipedia.org/wiki/Changi_Airport_MRT_station",
                         "https://en.wikipedia.org/wiki/Expo_MRT_station",
                         "https://en.wikipedia.org/wiki/Tanah_Merah_MRT_station"])

    stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_LRT_stations")
    stations.getTables()
    stationLinksL = stations.tables[0]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
    stationLinksL = [x for x in stationLinksL if ("a"+x)[-1] == "n"]
    stationLinksL.extend(["https://en.wikipedia.org/wiki/Damai_LRT_station_(Singapore)"])

    stationLinks = list(set(stationLinksM + stationLinksL))


    print("Extracting data from Wikipedia pages.")
    """
    PURPOSE

    Each page within the links has many key information scattered in various places.
    It was a challenge to make sure all the information from every page was captured, even if the data is inconsistently placed.
    The whole section below is to extract:
        Station Labels (NS26, EW14)
        Station Name (Raffles Place)
        Chinese Translation (莱佛士坊)
        Address (5 Raffles Place)
        Postal Code (048618)
        Latitude (Derived from 1°17′1.97″)
        Longitude (Derived from 103°51′5.52″)
    """
    mrt = []

    for link in tqdm(stationLinks):
        site = website(link)
        try:
            stationName = site.html("title")[0].string.split(" MRT")[0]
        except Exception as e:
            print("stationName gave the error: " + str(e) + "\n" + link)
            stationName = None

        try:
            stationDetails = list(site.html.find(class_="fn org").stripped_strings)
            chineseIndex = 0
            for detail in stationDetails:
                if re.compile('[\u4e00-\u9fff]+').search(detail):
                    chineseIndex = stationDetails.index(detail)
            stationLabels = ", ".join(stationDetails[:chineseIndex-1])
            stationChinese = stationDetails[chineseIndex]
        except Exception as e:
            print("stationLabels gave the error: " + str(e) + "\n" + link)
            stationLabels = None
            stationChinese = None

        try:
            lat = convertCoords(*re.findall("[0-9.]+", site.html(class_="latitude")[0].text))
            long = convertCoords(*re.findall("[0-9.]+", site.html(class_="longitude")[0].text))
        except Exception as e:
            print("latlong gave the error: " + str(e) + "\n" + link)
            lat = None
            long = None

        try:
            fullAddress = list(site.html(class_="infobox-data")[0].strings)
            address = fullAddress[0]
            postcode = re.findall(r"\d{6}", fullAddress[1])[0]
        except Exception as e:
            print("address gave the error: " + str(e) + "\n" + link)
            address = None
            postcode = None
        mrt.append({"Label": stationLabels, "Name": stationName, "Chinese": stationChinese, "Address": address, "Postcode": postcode, "Lat": lat, "Long": long, "Link": link})

grab.groupby("id").last().to_file("../Geospatial/Sheets/grab-end.geojson")
# grab_line_gdf.drop(["start_point","end_point"], axis=1).to_file("../Geospatial/Sheets/grab-line-computed.geojson")
grab_line_gdf.drop(["start_point","end_point"], axis=1).to_file("../Geospatial/Sheets/grab-line-computed.geojson")
# mp = grab_line_gdf.loc[list(grab_subset[grab_subset.plan.str.contains("MP")].index.unique())]
cg = grab_subset.loc[list(grab_subset[grab_subset.plan.str.contains("CHS")].index.unique())]
mrt_gdf = gpd.GeoDataFrame(mrt, geometry=gpd.points_from_xy(mrt.Long, mrt.Lat))
mrt_gdf.columns = ["label","abbr","name","chinese","o_year","c_year","address","postcode","lat","long","geometry"]
mrt_gdf.to_file("../Geospatial/Sheets/mrt.geojson")
planning = get_planning()
planning.columns = ["region","r_code","planning","p_code","subzone","s_code","geometry"]
planning_avg_speed = grab_subset.groupby("plan").speed.agg(np.mean).reset_index()
planning_grab = planning.merge(planning_avg_speed, left_on="s_code", right_on="plan")
gpd.GeoDataFrame(planning_grab, geometry="geometry").to_file("../Geospatial/Sheets/planning-grab.geojson")
# grab_subset["time"] = grab_subset.time.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
grab_subset["time"] = grab_subset.time.apply(lambda x: x.timestamp())
planning_avg_speed = grab_subset.groupby("plan").speed.agg(np.mean).reset_index()
grab_subset.to_file("../Geospatial/Sheets/grab-subset.geojson")
malls = malls[malls.Latitude > 0]
gpd.GeoDataFrame(malls, geometry=gpd.points_from_xy(malls.Longitude, malls.Latitude)).to_file("../Geospatial/Sheets/malls.geojson")
grab_subset.groupby("id").last().merge(grab_line_gdf[["duration", "s_distance", "t_efficiency"]], left_index=True, right_index=True, how="left").to_file("../Geospatial/Sheets/grab-end.geojson")
planning = get_planning()
planningm = planning[["PlanningCode", "SubzoneCode", "geometry"]].values.tolist()
planning_dict = planning.set_index("SubzoneCode").to_dict("index")
s_points = grab_line_gdf.start_point.tolist()
e_points = grab_line_gdf.end_point.tolist()

s_points_in_planning = []
for point in tqdm(s_points):
    not_done = True
    for p_code, s_code, geom in planningm:
        if point.within(geom):
            s_points_in_planning.append(s_code)
            not_done = False
            break
    if not_done:
        s_points_in_planning.append(np.nan)
        print("not ok")
        
e_points_in_planning = []
for point in tqdm(e_points):
    not_done = True
    for p_code, s_code, geom in planningm:
        if point.within(geom):
            e_points_in_planning.append(s_code)
            not_done = False
            break
    if not_done:
        e_points_in_planning.append(np.nan)
        print("not ok")
grab_line_gdf["start_plan"] = s_points_in_planning
grab_line_gdf["end_plan"] = e_points_in_planning
grab_line_plan = grab_line_gdf.groupby("start_plan").speed.agg(np.mean)
grab_line_plan = grab_line_plan.sort_values().reset_index()
grab_line_plan.columns = ["Planning Subzone", "Speed of journey"]
grab_line_plan["Planning Subzone"] = grab_line_plan["Planning Subzone"].apply(lambda x: planning_dict[x]["Planning"])
grab_line_plan["Speed of journey"] = grab_line_plan["Speed of journey"] / 60 * 1000
grab_line_gdf.drop(["start_point","end_point"],axis=1).to_file("../Geospatial/Sheets/grab-line-computed.geojson")
grab_line_gdf[grab_line_gdf.start_plan.str.contains("HGS")].end_plan.apply(lambda x: planning_dict[x]["Planning"]).value_counts()

# %store grab
# %store grab_line
# %store -r grab
# grab_copy = grab.copy()